<a href="https://colab.research.google.com/github/toktampazesh97/tamrin-datamining1/blob/main/pazesh(1)(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import numpy as np
import tensorflow as tf
import pandas as pd
from collections import Counter
import csv, re, pickle

fear = pd.read_excel("/content/pazeshh.xlsx")

In [11]:
fear =fear.dropna()
from sklearn.model_selection import train_test_split
train, test = train_test_split(fear, test_size=0.20, random_state=18)

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
import string
import re
import spacy
spacy.load('en')
from spacy.lang.en import English
parser = English()

In [13]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
SYMBOLS = " ".join(string.punctuation).split(" ") + ["-", "...", "”", "”"]
class CleanTextTransformer(TransformerMixin):   
  def transform(self, X, **transform_params):
        return [cleanText(text) for text in X]   
  def fit(self, X, y=None, **fit_params):
        return self
def get_params(self, deep=True):
        return {}
    
def cleanText(text):
    text = text.strip().replace("\n", " ").replace("\r", " ")
    text = text.lower()
    return text
def tokenizeText(sample):
    tokens = parser(sample)
    lemmas = []
    for tok in tokens:
        lemmas.append(tok.lemma_.lower().strip() if tok.lemma_ != "-PRON-" else tok.lower_)
    tokens = lemmas
    tokens = [tok for tok in tokens if tok not in STOPLIST]
    tokens = [tok for tok in tokens if tok not in SYMBOLS]
    return tokens

In [15]:
!pip install hazm
!pip install stopwords_guilannlp

In [16]:
import hazm 
from stopwords_guilannlp import *
normalizer =  hazm.Normalizer()
tokenizer = hazm.SentenceTokenizer()
tokenn=hazm.word_tokenize
STOPLIST=stopwords_output("Persian", "nar")

In [17]:

vectorizer = CountVectorizer(tokenizer=tokenn, ngram_range=(1,1))
#------------------------
clf = LinearSVC()

pipe = Pipeline([('cleanText', CleanTextTransformer()), ('vectorizer', vectorizer), ('clf', clf)])# data
train1 = train['comment'].tolist()
labelsTrain1 = train['Label'].tolist()
test1 = test['comment'].tolist()
labelsTest1 = test['Label'].tolist()
# train
pipe.fit(train1, labelsTrain1)# test
preds = pipe.predict(test1)
print("accuracy:", accuracy_score(labelsTest1, preds))

pipe = Pipeline([('cleanText', CleanTextTransformer()), ('vectorizer', vectorizer)])
transform = pipe.fit_transform(train1, labelsTrain1)
vocab = vectorizer.get_feature_names()
for i in range(len(train1)):
    s = ""
    indexIntoVocab = transform.indices[transform.indptr[i]:transform.indptr[i+1]]
    numOccurences = transform.data[transform.indptr[i]:transform.indptr[i+1]]
    for idx, num in zip(indexIntoVocab, numOccurences):
        s += str((vocab[idx], num))

accuracy: 0.495


In [18]:
from sklearn import metrics
print(metrics.classification_report(labelsTest1, preds, 
                                    target_names=fear['Label'].unique()))

              precision    recall  f1-score   support

   U0001F49C       0.50      0.33      0.40         6
   U0001F60D       0.40      0.48      0.43        42
   U0001F49F       0.51      0.61      0.56        70
   U0001F928       0.00      0.00      0.00        12
   U0001F61F       0.55      0.62      0.58       109
   U0001F613       0.27      0.25      0.26        36
   U0001F912       0.14      0.06      0.08        17
   U0001F4A8       0.59      0.71      0.65        45
   U0001F63E       0.38      0.14      0.20        22
   U0001F479       0.56      0.49      0.52        41

    accuracy                           0.49       400
   macro avg       0.39      0.37      0.37       400
weighted avg       0.46      0.49      0.47       400



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
